In [2]:
import cv2
import zmq
import time
import torch
import pickle
import carla
import numpy as np
import gymnasium as gym
from process_frame import process_frame
from stable_baselines3 import PPO


In [2]:
context = zmq.Context()
socket = context.socket(zmq.REQ)
socket.connect("tcp://localhost:6501")

def send_action(steer: float, throttle: float):
    action = {
        "action": [steer, throttle]
    }
    socket.send(pickle.dumps(action))  # Serialize & send
    response = socket.recv()           # Receive serialized reply
    data = pickle.loads(response)      # Deserialize

    return data


# ==== Decode image from bytes ====
def decode_image(image_bytes):
    np_arr = np.frombuffer(image_bytes, dtype=np.uint8)
    img = cv2.imdecode(np_arr, cv2.IMREAD_COLOR)
    return img

# ==== Main loop ====
for step in range(200):
    steer = 0
    throttle =0
    obs = send_action(steer, throttle)

    # Extract observation
    # try:
    image = decode_image(obs["image"])
    gnss = obs["gnss"]
    collision = obs["collision"]
    imu = obs["imu"]
    timestamp = obs["timestamp"]
    lane_invaded = obs["lane_invaded"]
    
    procsses_image = process_frame(image)
    
    if cv2.waitKey(1) == ord('q'):
        quit = True
        break
    
    # print(f"[Step {step}] GNSS: {gnss} | Time: {timestamp} | Collision: {collision} | imu: {imu} | lane: {lane_invaded}")
    # # print(f"lane: {lane_invaded}")
    # if lane_invaded['violated'] == True:
    #     print(lane_invaded["last_event"])
    #     print(lane_invaded)
        # cv2.imshow("Invasion", image)

    # Optional: show image
    cv2.imshow("Camera", procsses_image)
    cv2.waitKey(1)

    # except Exception as e:
    #     print(lane_invaded["last_event"])
    #     print(lane_invaded)
    #     print(e)
    #     # print(f"[ERROR] Failed to process observation: {e}")
    #     # print(f"[DEBUG] Image shape before processing: {image.shape}")
    #     pass
    #     # break

    time.sleep(0.05)  # slow it down for now
cv2.destroyAllWindows()

# Reset Carla at end (optional)
# reset_command = {"command": "reset"}
# socket.send(pickle.dumps(reset_command))
# _ = socket.recv()